In [10]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import scipy
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

In [11]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('./archive/Data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./archive/Data/features_30_sec.csv
./archive/Data/features_3_sec.csv
./archive/Data/genres_original\blues\blues.00000.wav
./archive/Data/genres_original\blues\blues.00001.wav
./archive/Data/genres_original\blues\blues.00002.wav
./archive/Data/genres_original\blues\blues.00003.wav
./archive/Data/genres_original\blues\blues.00004.wav
./archive/Data/genres_original\blues\blues.00005.wav
./archive/Data/genres_original\blues\blues.00006.wav
./archive/Data/genres_original\blues\blues.00007.wav
./archive/Data/genres_original\blues\blues.00008.wav
./archive/Data/genres_original\blues\blues.00009.wav
./archive/Data/genres_original\blues\blues.00010.wav
./archive/Data/genres_original\blues\blues.00011.wav
./archive/Data/genres_original\blues\blues.00012.wav
./archive/Data/genres_original\blues\blues.00013.wav
./archive/Data/genres_original\blues\blues.00014.wav
./archive/Data/genres_original\blues\blues.00015.wav
./archive/Data/genres_original\blues\blues.00016.wav
./archive/Data/genres_original

In [12]:
df = pd.read_csv("./archive/Data/features_3_sec.csv")
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [13]:
df=df.drop(labels="filename",axis=1)

In [14]:
class_list=df.iloc[:,-1]
converter=LabelEncoder()

y=converter.fit_transform(class_list)
print(y)

print(df.iloc[:,:-1])



[0 0 0 ... 9 9 9]
      length  chroma_stft_mean  chroma_stft_var  rms_mean   rms_var  \
0      66149          0.335406         0.091048  0.130405  0.003521   
1      66149          0.343065         0.086147  0.112699  0.001450   
2      66149          0.346815         0.092243  0.132003  0.004620   
3      66149          0.363639         0.086856  0.132565  0.002448   
4      66149          0.335579         0.088129  0.143289  0.001701   
...      ...               ...              ...       ...       ...   
9985   66149          0.349126         0.080515  0.050019  0.000097   
9986   66149          0.372564         0.082626  0.057897  0.000088   
9987   66149          0.347481         0.089019  0.052403  0.000701   
9988   66149          0.387527         0.084815  0.066430  0.000320   
9989   66149          0.369293         0.086759  0.050524  0.000067   

      spectral_centroid_mean  spectral_centroid_var  spectral_bandwidth_mean  \
0                1773.065032          167541.6308

## Normalization

In [15]:
from sklearn.preprocessing import StandardScaler
fit=StandardScaler()
X=fit.fit_transform(np.array(df.iloc[:,:-1],dtype=float))

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)

## Model

In [17]:
from tensorflow.keras.models import Sequential
import tensorflow as tf
def trainModel(model,epochs,optimizer):
    batch_size=128
    model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics='accuracy')
    return model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=epochs,batch_size=batch_size)

def plotValidate(history):
    print("Validation Accuracy",max(history.history["val_accuracy"]))
    pd.DataFrame(history.history).plot(figsize=(12,6))
    plt.show()

In [18]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(512,activation='relu',input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256,activation='relu'),
    keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(10,activation='softmax'),
])

print(model.summary())
model_history=trainModel(model=model,epochs=600,optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               30208     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8

## Evaluation

In [19]:
test_loss,test_acc=model.evaluate(X_test,y_test,batch_size=128)
print("The test loss is ",test_loss)
print("The best accuracy is: ",test_acc*100)

26/26 [==============================] - 0s 1ms/step - loss: 0.4445 - accuracy: 0.9311
The test loss is  0.4445470869541168
The best accuracy is:  93.11495423316956


## Demo

In [27]:
example = X[0]
example = np.expand_dims(example, axis=0)

In [28]:
model.predict(example)

1/1 [==============================] - 0s 66ms/step


array([[1.0000000e+00, 1.4817452e-12, 5.7098459e-10, 1.2532046e-08,
        2.9469092e-08, 3.7257450e-09, 8.3134145e-11, 3.4003161e-13,
        9.9328135e-10, 4.6186280e-08]], dtype=float32)

In [39]:
X.shape

(9990, 58)

In [40]:
example = X[1001]
example = np.expand_dims(example, axis=0)

predictions = model.predict(example)

predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 14ms/step
Predicted class: [1]


In [41]:
example = X[2001]
example = np.expand_dims(example, axis=0)

predictions = model.predict(example)

predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 14ms/step
Predicted class: [2]
